# Decision Tree
- ref: https://medium.com/@penggongting/implementing-decision-tree-from-scratch-in-python-c732e7c69aea

## Basic function

In [1]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import math

In [2]:
# entropy 
def entropy(c, n):
    return -(c/n)*math.log(c/n, 2)
    # gini index
    # return 1-(c/n)**2

# Split into two categories based on feature values, calculate entropy for each category, and sum them
def entropy_cal(c1, c2):
    if c1== 0 or c2 == 0: # avoid log(0) and if c1==0, so c2==n, entropy(c2) = -(c2/n)*log(c2/n) = 0
        return 0
    return entropy(c1, c1+c2) + entropy(c2, c1+c2)

# Split the data based on each category variable, calculate entropy for each category, and sum them
def entropy_of_one_division(division):
    s = 0
    n = len(division)
    classes = set(division)
    # Calculate the entropy for each category and sum them
    for c in classes:   
        n_c = sum(division==c)
        e = n_c/n * entropy_cal(sum(division==c), sum(division!=c))
        s += e
    return s, n

# Calculate entropy based on the splitting condition
def get_entropy(y_predict, y_real):
    if len(y_predict) != len(y_real):
        print('They have to be the same length')
        return None
    n = len(y_real)
    # left node
    s_true, n_true = entropy_of_one_division(y_real[y_predict])
    # right node
    s_false, n_false = entropy_of_one_division(y_real[~y_predict])
    # Weighted sum of left and right nodes
    s = n_true/n * s_true + n_false/n * s_false 
    return s

## Decision tree algorithm

In [3]:
class DecisionTreeClassifier(object):
    def __init__(self, max_depth=3):
        self.depth = 0
        self.max_depth = max_depth

    # train the model
    def fit(self, x, y, par_node={}, depth=0):
        if par_node is None: 
            return None
        elif len(y) == 0:
            return None
        elif self.all_same(y):
            return {'val':float(y[0])}
        elif depth >= self.max_depth:
            return None
        else: 
            # calculate the information gain
            col, cutoff, entropy = self.find_best_split_of_all(x, y)
            if cutoff is not None:
                y_left = y[x[:, col] < cutoff]
                y_right = y[x[:, col] >= cutoff]
                par_node = {'col': feature_names[col], 'index_col':int(col),
                            'cutoff':float(cutoff),
                            'val': float(np.round(np.mean(y)))}
                par_node['left'] = self.fit(x[x[:, col] < cutoff], y_left, {}, depth+1)
                par_node['right'] = self.fit(x[x[:, col] >= cutoff], y_right, {}, depth+1)
                self.depth += 1 
            self.trees = par_node
            return par_node


    # compare all the feature value and the threshold to find the best split point
    def find_best_split_of_all(self, x, y):
        col = None
        min_entropy = 1
        cutoff = None
        for i, c in enumerate(x.T):
            entropy, cur_cutoff = self.find_best_split(c, y)
            if entropy == 0:    # find the best split point
                return i, cur_cutoff, entropy
            elif entropy <= min_entropy:
                min_entropy = entropy
                col = i
                cutoff = cur_cutoff
        return col, cutoff, min_entropy
    
    # find the best split point
    def find_best_split(self, col, y):
        min_entropy = 10
        n = len(y)
        for value in set(col):
            y_predict = col < value
            my_entropy = get_entropy(y_predict, y)
            if my_entropy < min_entropy:
                min_entropy = my_entropy
                cutoff = value
        return min_entropy, cutoff
    
    # check if all the values are the same in the node
    def all_same(self, items):
        return all(x == items[0] for x in items)
    
    # predict the value
    def predict(self, x):
        tree = self.trees
        results = np.array([0]*len(x))
        for i, c in enumerate(x):
            try:
                results[i] = self._get_prediction(c)
            except:
                pass
        return results
    
    # get the prediction value
    def _get_prediction(self, row):
        cur_layer = self.trees
        while cur_layer is not None and cur_layer.get('cutoff'):
            if row[cur_layer['index_col']] < cur_layer['cutoff']:
                cur_layer = cur_layer['left']
            else:
                cur_layer = cur_layer['right']
        else:
            return cur_layer.get('val') if cur_layer else None
    

## Load data

In [4]:
ds = datasets.load_wine()
feature_names = ds.feature_names
X, y = ds.data, ds.target

In [5]:
X

array([[1.423e+01, 1.710e+00, 2.430e+00, ..., 1.040e+00, 3.920e+00,
        1.065e+03],
       [1.320e+01, 1.780e+00, 2.140e+00, ..., 1.050e+00, 3.400e+00,
        1.050e+03],
       [1.316e+01, 2.360e+00, 2.670e+00, ..., 1.030e+00, 3.170e+00,
        1.185e+03],
       ...,
       [1.327e+01, 4.280e+00, 2.260e+00, ..., 5.900e-01, 1.560e+00,
        8.350e+02],
       [1.317e+01, 2.590e+00, 2.370e+00, ..., 6.000e-01, 1.620e+00,
        8.400e+02],
       [1.413e+01, 4.100e+00, 2.740e+00, ..., 6.100e-01, 1.600e+00,
        5.600e+02]])

In [6]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2])

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

## Model training

In [8]:
feature_names

['alcohol',
 'malic_acid',
 'ash',
 'alcalinity_of_ash',
 'magnesium',
 'total_phenols',
 'flavanoids',
 'nonflavanoid_phenols',
 'proanthocyanins',
 'color_intensity',
 'hue',
 'od280/od315_of_diluted_wines',
 'proline']

In [9]:
import json

clf = DecisionTreeClassifier()
output = clf.fit(X_train, y_train)
# output
print(json.dumps(output, indent=4))

{
    "col": "color_intensity",
    "index_col": 9,
    "cutoff": 3.52,
    "val": 1.0,
    "left": {
        "val": 1.0
    },
    "right": {
        "col": "proline",
        "index_col": 12,
        "cutoff": 845.0,
        "val": 1.0,
        "left": {
            "col": "flavanoids",
            "index_col": 6,
            "cutoff": 1.25,
            "val": 2.0,
            "left": {
                "val": 2.0
            },
            "right": null
        },
        "right": {
            "val": 0.0
        }
    }
}


In [10]:
# 計算準確率
y_pred = clf.predict(X_test)
print(f'{accuracy_score(y_test, y_pred)*100:.2f}%') 

83.33%
